In [0]:
# ==============================================================================
# 1. IMPORT LIBRATIES
# ==============================================================================
import pandas as pd
import numpy as np
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
import os
from pyspark.sql import SparkSession

In [0]:
# ==============================================================================
# 2. CONFIGURATION AND UTILS
# ==============================================================================

INPUT_TABLE_NAME = "retail_feature_engg_done"

# --- DATABASE CONNECTION (Assumes Databricks Secrets/dbutils) ---
# Replace this with your secure connection logic
pg_host = dbutils.secrets.get(scope="postgres-secrets", key="host")
pg_port = dbutils.secrets.get(scope="postgres-secrets", key="port")
pg_database = dbutils.secrets.get(scope="postgres-secrets", key="database")
pg_username = dbutils.secrets.get(scope="postgres-secrets", key="username")
pg_password = dbutils.secrets.get(scope="postgres-secrets", key="password")

jdbc_url = f"jdbc:postgresql://{pg_host}:{pg_port}/{pg_database}"
CONNECTION_PROPERTIES = {
    "user": pg_username,
    "password": pg_password,
    "driver": "org.postgresql.Driver" }



In [0]:
# ============================================================
# 3. RFM CALCULATION
# ============================================================

def calculate_rfm(table_name: str, spark: SparkSession, analysis_date=None) -> pd.DataFrame:
    """
    Calculate RFM metrics for each customer
    """
    print(f"Reading data from PostgreSQL table: {table_name}...")
    
    spark_df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=CONNECTION_PROPERTIES)
    df = spark_df.toPandas()
    
    df['purchase_datetime'] = pd.to_datetime(df['purchase_datetime'])
    
    # Use most recent date in data as analysis date
    if analysis_date is None:
        analysis_date = df['purchase_datetime'].max()
    
    print(f"✓ Loaded {len(df):,} transactions")
    print(f"✓ Date range: {df['purchase_datetime'].min()} to {df['purchase_datetime'].max()}")
    print(f"✓ Analysis date: {analysis_date}")
    
    # Calculate RFM metrics
    rfm = df.groupby('customer_key').agg({
        'purchase_datetime': lambda x: (analysis_date - x.max()).days,  # Recency
        'order_id': 'nunique',  # Frequency
        'total_payment_value': 'sum'  # Monetary
    }).reset_index()
    
    rfm.columns = ['customer_key', 'recency', 'frequency', 'monetary']
    
    # Add additional useful metrics
    first_purchase = df.groupby('customer_key')['purchase_datetime'].min().reset_index()
    first_purchase.columns = ['customer_key', 'first_purchase_date']
    rfm = rfm.merge(first_purchase, on='customer_key')
    rfm['customer_age_days'] = (analysis_date - rfm['first_purchase_date']).dt.days
    
    # Average order value
    rfm['avg_order_value'] = rfm['monetary'] / rfm['frequency']
    
    print(f"\n✓ RFM Metrics Calculated for {len(rfm):,} customers")
    print(f"\nRFM Summary Statistics:")
    print(rfm[['recency', 'frequency', 'monetary']].describe())
    
    return rfm, analysis_date

In [0]:
# ============================================================
# 4. RFM SCORING (Traditional Approach)
# ============================================================

def create_rfm_scores(rfm: pd.DataFrame) -> pd.DataFrame:
    """
    Create RFM scores (1-5) for each metric using quintiles
    Lower recency is better, higher frequency/monetary is better
    """
    rfm_scored = rfm.copy()
    
    # Recency Score (1-5, where 5 is best/most recent)
    rfm_scored['R_score'] = pd.qcut(rfm_scored['recency'], q=5, labels=[5,4,3,2,1], duplicates='drop')
    
    # Frequency Score (1-5, where 5 is best/most frequent)
    rfm_scored['F_score'] = pd.qcut(rfm_scored['frequency'].rank(method='first'), q=5, labels=[1,2,3,4,5], duplicates='drop')
    
    # Monetary Score (1-5, where 5 is best/highest value)
    rfm_scored['M_score'] = pd.qcut(rfm_scored['monetary'].rank(method='first'), q=5, labels=[1,2,3,4,5], duplicates='drop')
    
    # Convert to int
    rfm_scored['R_score'] = rfm_scored['R_score'].astype(int)
    rfm_scored['F_score'] = rfm_scored['F_score'].astype(int)
    rfm_scored['M_score'] = rfm_scored['M_score'].astype(int)
    
    # Combined RFM Score
    rfm_scored['RFM_score'] = (rfm_scored['R_score'].astype(str) + 
                               rfm_scored['F_score'].astype(str) + 
                               rfm_scored['M_score'].astype(str))
    
    # Overall score (simple average)
    rfm_scored['RFM_total'] = rfm_scored['R_score'] + rfm_scored['F_score'] + rfm_scored['M_score']
    
    return rfm_scored

In [0]:
# ============================================================
# 5. CUSTOMER SEGMENTATION
# ============================================================

def segment_customers(rfm_scored: pd.DataFrame) -> pd.DataFrame:
    """
    Assign customers to business-friendly segments based on RFM scores
    """
    rfm_segmented = rfm_scored.copy()
    
    def assign_segment(row):
        r, f, m = row['R_score'], row['F_score'], row['M_score']
        
        # Champions: Best customers - bought recently, buy often, spend the most
        if r >= 4 and f >= 4 and m >= 4:
            return 'Champions'
        
        # Loyal: Frequent buyers, but not highest spenders
        elif r >= 3 and f >= 4:
            return 'Loyal'
        
        # Potential Loyalists: Recent customers with average frequency
        elif r >= 4 and f >= 2 and f <= 3:
            return 'Potential Loyalist'
        
        # Recent Customers: Bought recently but not frequent
        elif r >= 4 and f == 1:
            return 'New Customer'
        
        # Promising: Recent buyers with low frequency
        elif r >= 3 and f == 1:
            return 'Promising'
        
        # Need Attention: Above average recency, frequency, and monetary
        elif r >= 3 and f >= 2 and m >= 3:
            return 'Need Attention'
        
        # At Risk: Made big purchases but haven't returned
        elif r <= 2 and f >= 2 and m >= 3:
            return 'At Risk'
        
        # Can't Lose Them: Used to spend big, but slipping away
        elif r <= 2 and f >= 4 and m >= 4:
            return 'Cannot Lose'
        
        # Hibernating: Last purchase was long ago, low spenders
        elif r <= 2 and f <= 2 and m <= 2:
            return 'Hibernating'
        
        # Lost: Lowest recency, frequency, and monetary scores
        elif r == 1:
            return 'Lost'
        
        else:
            return 'Others'
    
    rfm_segmented['segment'] = rfm_segmented.apply(assign_segment, axis=1)
    
    return rfm_segmented

In [0]:
# ============================================================
# 6. K-MEANS CLUSTERING (ML Approach)
# ============================================================

import sys, os

def kmeans_segmentation(rfm: pd.DataFrame, n_clusters=5) -> pd.DataFrame:
    rfm_clustering = rfm[['recency', 'frequency', 'monetary']].copy()
    rfm_clustering['frequency_log'] = np.log1p(rfm_clustering['frequency'])
    rfm_clustering['monetary_log'] = np.log1p(rfm_clustering['monetary'])

    scaler = StandardScaler()
    rfm_scaled = scaler.fit_transform(rfm_clustering[['recency','frequency_log','monetary_log']])
    
    # Suppress stderr for the KMeans fit
    stderr_fileno = sys.stderr
    sys.stderr = open(os.devnull, 'w')
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    rfm['cluster'] = kmeans.fit_predict(rfm_scaled)
    
    sys.stderr.close()
    sys.stderr = stderr_fileno

    cluster_summary = rfm.groupby('cluster').agg({
        'recency':'mean','frequency':'mean','monetary':'mean','customer_key':'count'
    }).round(2)
    cluster_summary.columns = ['avg_recency','avg_frequency','avg_monetary','customer_count']
    
    print(f"\n✓ K-Means Clustering Summary (n_clusters={n_clusters}):")
    print(cluster_summary)
    
    return rfm, kmeans, scaler

In [0]:
# ============================================================
# 7. VISUALIZATION
# ============================================================

def create_rfm_visualizations(rfm_segmented: pd.DataFrame, save_path='rfm_analysis.png'):
    """
    Create comprehensive RFM visualizations
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 5a. Segment Distribution
    segment_counts = rfm_segmented['segment'].value_counts()
    axes[0, 0].barh(range(len(segment_counts)), segment_counts.values, color='skyblue')
    axes[0, 0].set_yticks(range(len(segment_counts)))
    axes[0, 0].set_yticklabels(segment_counts.index)
    axes[0, 0].set_xlabel('Number of Customers')
    axes[0, 0].set_title('Customer Segments Distribution', fontsize=14, fontweight='bold')
    axes[0, 0].invert_yaxis()
    
    # 5b. Revenue by Segment
    revenue_by_segment = rfm_segmented.groupby('segment')['monetary'].sum().sort_values(ascending=True)
    axes[0, 1].barh(range(len(revenue_by_segment)), revenue_by_segment.values, color='green', alpha=0.7)
    axes[0, 1].set_yticks(range(len(revenue_by_segment)))
    axes[0, 1].set_yticklabels(revenue_by_segment.index)
    axes[0, 1].set_xlabel('Total Revenue ($)')
    axes[0, 1].set_title('Revenue by Customer Segment', fontsize=14, fontweight='bold')
    axes[0, 1].invert_yaxis()
    
    # 5c. RFM Score Distribution
    axes[0, 2].hist(rfm_segmented['RFM_total'], bins=range(3, 16), edgecolor='black', alpha=0.7, color='coral')
    axes[0, 2].set_xlabel('RFM Total Score')
    axes[0, 2].set_ylabel('Number of Customers')
    axes[0, 2].set_title('RFM Score Distribution', fontsize=14, fontweight='bold')
    axes[0, 2].grid(True, alpha=0.3, axis='y')
    
    # 5d. Recency vs Frequency Scatter (colored by segment)
    top_segments = rfm_segmented['segment'].value_counts().head(5).index
    for segment in top_segments:
        segment_data = rfm_segmented[rfm_segmented['segment'] == segment]
        axes[1, 0].scatter(segment_data['recency'], segment_data['frequency'], 
                          alpha=0.6, s=30, label=segment)
    axes[1, 0].set_xlabel('Recency (days)')
    axes[1, 0].set_ylabel('Frequency (orders)')
    axes[1, 0].set_title('Recency vs Frequency by Segment', fontsize=14, fontweight='bold')
    axes[1, 0].legend(fontsize=8)
    axes[1, 0].grid(True, alpha=0.3)
    
    # 5e. Frequency vs Monetary Scatter
    for segment in top_segments:
        segment_data = rfm_segmented[rfm_segmented['segment'] == segment]
        axes[1, 1].scatter(segment_data['frequency'], segment_data['monetary'], 
                          alpha=0.6, s=30, label=segment)
    axes[1, 1].set_xlabel('Frequency (orders)')
    axes[1, 1].set_ylabel('Monetary ($)')
    axes[1, 1].set_title('Frequency vs Monetary by Segment', fontsize=14, fontweight='bold')
    axes[1, 1].legend(fontsize=8)
    axes[1, 1].grid(True, alpha=0.3)
    
    # 5f. Average Order Value by Segment
    aov_by_segment = rfm_segmented.groupby('segment')['avg_order_value'].mean().sort_values(ascending=True)
    axes[1, 2].barh(range(len(aov_by_segment)), aov_by_segment.values, color='purple', alpha=0.7)
    axes[1, 2].set_yticks(range(len(aov_by_segment)))
    axes[1, 2].set_yticklabels(aov_by_segment.index)
    axes[1, 2].set_xlabel('Average Order Value ($)')
    axes[1, 2].set_title('AOV by Customer Segment', fontsize=14, fontweight='bold')
    axes[1, 2].invert_yaxis()
    
    plt.tight_layout(pad=3.0)
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"\n✓ Visualization saved to: {save_path}")
    plt.close(fig)

In [0]:
# ============================================================
# 8. MAIN PIPELINE
# ============================================================

def main_rfm_run(input_table):
    """
    Main RFM segmentation pipeline
    """
    with mlflow.start_run(run_name="RFM_Customer_Segmentation") as run:
        
        # Log parameters
        mlflow.log_params({
            "model_type": "RFM Segmentation",
            "segmentation_method": "Rule-based + K-Means"
        })
        
        # 1. Calculate RFM
        rfm, analysis_date = calculate_rfm(input_table, spark)
        
        # 2. Create RFM Scores
        rfm_scored = create_rfm_scores(rfm)
        
        # 3. Segment Customers
        rfm_segmented = segment_customers(rfm_scored)
        
        # 4. K-Means Alternative
        rfm_kmeans, kmeans_model, scaler = kmeans_segmentation(rfm.copy(), n_clusters=5)
        
        # 5. Segment Analysis
        print(f"\n{'='*60}")
        print("SEGMENT ANALYSIS")
        print(f"{'='*60}")
        
        segment_analysis = rfm_segmented.groupby('segment').agg({
            'customer_key': 'count',
            'recency': 'mean',
            'frequency': 'mean',
            'monetary': ['mean', 'sum'],
            'avg_order_value': 'mean'
        }).round(2)
        
        segment_analysis.columns = ['customer_count', 'avg_recency', 'avg_frequency', 
                                   'avg_monetary', 'total_revenue', 'avg_order_value']
        segment_analysis = segment_analysis.sort_values('total_revenue', ascending=False)
        
        print(segment_analysis)
        
        # Calculate segment percentages
        total_customers = len(rfm_segmented)
        total_revenue = rfm_segmented['monetary'].sum()
        
        for segment in segment_analysis.index:
            seg_customers = segment_analysis.loc[segment, 'customer_count']
            seg_revenue = segment_analysis.loc[segment, 'total_revenue']
            print(f"\n{segment}:")
            print(f"  {seg_customers} customers ({seg_customers/total_customers*100:.1f}%)")
            print(f"  ${seg_revenue:,.2f} revenue ({seg_revenue/total_revenue*100:.1f}%)")
        
        # 6. Log metrics
        mlflow.log_metrics({
            "total_customers": total_customers,
            "total_revenue": total_revenue,
            "num_segments": rfm_segmented['segment'].nunique(),
            "champions_pct": (rfm_segmented['segment'] == 'Champions').mean() * 100,
            "at_risk_pct": (rfm_segmented['segment'] == 'At Risk').mean() * 100
        })
        
        # 7. Save outputs
        output_path = 'rfm_customer_segments.csv'
        rfm_segmented.to_csv(output_path, index=False)
        mlflow.log_artifact(output_path)
        
        # 8. Create visualizations
        create_rfm_visualizations(rfm_segmented)
        mlflow.log_artifact('rfm_analysis.png')
        
        print(f"\n✨ RFM Segmentation Complete!")
        
        return rfm_segmented

In [0]:
# ============================================================
# 9. EXECUTION
# ============================================================

rfm_results = main_rfm_run(INPUT_TABLE_NAME)
display(rfm_results.head(20))

Reading data from PostgreSQL table: retail_feature_engg_done...
✓ Loaded 104,023 transactions
✓ Date range: 2016-09-04 21:15:00 to 2018-09-03 09:06:00
✓ Analysis date: 2018-09-03 09:06:00

✓ RFM Metrics Calculated for 93,395 customers

RFM Summary Statistics:
            recency     frequency      monetary
count  93395.000000  93395.000000  93395.000000
mean     241.740629      1.033406    176.016423
std      153.161514      0.208236    265.079187
min        0.000000      1.000000      8.860000
25%      118.000000      1.000000     63.120000
50%      222.000000      1.000000    108.760000
75%      351.000000      1.000000    188.500000
max      728.000000     15.000000  15978.650000

✓ K-Means Clustering Summary (n_clusters=5):
         avg_recency  avg_frequency  avg_monetary  customer_count
cluster                                                          
0             133.87           1.00        163.88           28459
1             427.32           1.00        106.61           2543

customer_key,recency,frequency,monetary,first_purchase_date,customer_age_days,avg_order_value,R_score,F_score,M_score,RFM_score,RFM_total,segment
0000366f3b9a7992bf8c76cfdf3221e2,115,1,141.9,2018-05-10T10:56:00Z,115,141.9,4,1,4,414,9,New Customer
0000b849f77a49e4a4ce2b2a4ca5be3f,118,1,27.19,2018-05-07T11:11:00Z,118,27.19,4,1,1,411,6,New Customer
0000f46a3911fa3c0805444483337064,541,1,86.22,2017-03-10T21:05:00Z,541,86.22,1,1,2,112,4,Hibernating
0000f6ccb0745a6a4b88665a16c9f078,325,1,43.62,2017-10-12T20:29:00Z,325,43.62,2,1,1,211,4,Hibernating
0004aac84e0df4da2b147fca70cf8255,292,1,196.89,2017-11-14T19:45:00Z,292,196.89,2,1,4,214,7,Others
0004bd2a26a76fe21f786e4fbd80607f,150,1,166.98,2018-04-05T19:33:00Z,150,166.98,4,1,4,414,9,New Customer
00050ab1314c0e55a6ca13cf7181fecf,135,1,35.38,2018-04-20T12:57:00Z,135,35.38,4,1,1,411,6,New Customer
00053a61a98854899e70ed204dd4bafe,186,1,838.36,2018-02-28T11:15:00Z,186,838.36,3,1,5,315,9,Promising
0005e1862207bf6ccc02e4228effd9a0,547,1,150.12,2017-03-04T23:32:00Z,547,150.12,1,1,4,114,6,Lost
0005ef4cd20d2893f0d9fbd94d3c0d97,174,1,129.76,2018-03-12T15:22:00Z,174,129.76,4,1,3,413,8,New Customer
